In [3]:
from load_config import read_grinding_config
import numpy as np

## function definition

In [4]:
def selection_function_basic(
    d_i_star: float, 
    a0: float, 
    a1: float, 
    d_crit: float, 
    a2: float
) -> float:
    """
    计算基础选择函数 S_i (Equation 6)
    
    Args:
        d_i_star: 平均粒径 (di* = sqrt(di * di+1))
        a0: 缩放系数 alpha0
        a1: 指数系数 alpha1
        d_crit: 临界粒径 dcrit
        a2: 指数系数 alpha2
    
    Returns:
        选择函数值 S_i
    """
    numerator = a0 * (d_i_star ** a1)
    denominator = 1 + (d_i_star / d_crit) ** a2
    return numerator / denominator


def calculate_d_i_star(d_i: float, d_i_plus_1: float) -> float:
    """
    计算平均粒径 di* = sqrt(di * di+1)
    """
    return np.sqrt(d_i * d_i_plus_1)

def selection_function_expanded(
    d_i_star: float,
    a01: float,
    a11: float,
    a02: float,
    a12: float,
    d_crit: float,
    a2: float
) -> float:
    """
    计算扩展选择函数 S_i (Equation 6e)
    
    Args:
        d_i_star: 平均粒径
        a01: 扩展参数 alpha01
        a11: 扩展参数 alpha11
        a02: 扩展参数 alpha02
        a12: 扩展参数 alpha12
        d_crit: 临界粒径
        a2: 指数系数 alpha2
    
    Returns:
        扩展选择函数值 S_i
    """
    term1_numerator = a01 * (d_i_star ** a11)
    term1_denominator = 1 + (d_i_star / d_crit) ** a2
    term1 = term1_numerator / term1_denominator
    
    term2 = a02 * (d_i_star ** a12)
    
    scaling_factor = 1 / (1 + a02 / a01) if a01 != 0 else 0
    
    return scaling_factor * (term1 + term2)

def breakage_function_basic(
    d_i: float,
    d_j_plus_1: float,
    b0: float,
    b1: float,
    b2: float
) -> float:
    """
    计算基础破碎函数 B_ij (Equation 7)
    
    Args:
        d_i: 原始粒径 di
        d_j_plus_1: 下一粒径 dj+1
        b0: 权重系数 beta0
        b1: 指数系数 beta1
        b2: 指数系数 beta2
    
    Returns:
        破碎函数值 B_ij
    """
    ratio = d_i / d_j_plus_1
    return b0 * (ratio ** b1) + (1 - b0) * (ratio ** b2)

def calculate_beta0j(
    d_j_plus_1: float,  # 单位：μm
    beta00: float,      # 对应论文中的β₀₀
    beta01: float,      # 对应论文中的β₀₁
    max_limit: float = 1.0
) -> float:
    """
    计算动态破碎权重系数 β₀ⱼ (Equation below 7 in original paper)
    
    Args:
        d_j_plus_1: 下一粒径 d_{j+1} (单位：微米μm)
        beta00: 基础系数 (无量纲)
        beta01: 衰减指数 (无量纲)
        max_limit: β₀ⱼ的最大值限制 (默认1.0)
    
    Returns:
        动态权重系数 β₀ⱼ ∈ [0, max_limit]
    
    Notes:
        1. 公式原型：β₀ⱼ = β₀₀·(d_{j+1}/1000)^{-β₀₁}
        2. 隐含单位转换：分母1000将μm转换为mm
        3. 根据论文要求，β₀ⱼ不得超过1.0
    """
    beta0j = beta00 * (d_j_plus_1 / 1000) ** (-beta01)
    return min(beta0j, max_limit)

def breakage_function_expanded(
    d_i: float,          # 原始粒径 d_i (μm)
    d_j_plus_1: float,   # 下一粒径 d_{j+1} (μm)
    beta00: float,       # 对应论文参数
    beta01: float,       # 对应论文参数
    beta1: float,       # 对应论文中的β₁
    beta2: float        # 对应论文中的β₂
) -> float:
    """
    计算扩展破碎函数 B_{ij} (Equation 7 with dynamic β₀ⱼ)
    
    Args:
        d_i: 原始粒径 (μm)
        d_j_plus_1: 参考粒径 (μm)
        beta00: 基础权重系数
        beta01: 粒径衰减指数
        beta1: 粗粒组分指数
        beta2: 细粒组分指数
    
    Returns:
        破碎概率 B_{ij} ∈ [0,1]
    """
    beta0j = calculate_beta0j(d_j_plus_1, beta00, beta01)
    # print(beta0j)
    ratio = d_i / d_j_plus_1
    return beta0j * (ratio ** beta1) + (1 - beta0j) * (ratio ** beta2)

In [5]:
def calculate_slurry_density(
    percent_solids: float,  # 固体百分比（0-100%）
    ore_density: float,     # 固体密度（ton/m³）
    water_density: float = 1.00  # 水的密度（ton/m³），默认1.0
) -> float:
    """
    计算矿浆密度（Slurry Density）
    
    公式：
    ρ_slurry = 1 / [ (w_solids/ρ_ore) + (w_water/ρ_water) ]
    其中 w_solids = %_solids / 100, w_water = 1 - w_solids

    Args:
        percent_solids: 固体质量百分比（%）
        ore_density: 固体密度（ton/m³）
        water_density: 液体密度（ton/m³），默认为1.0（水）

    Returns:
        矿浆密度（ton/m³）

    Raises:
        ValueError: 如果输入百分比或密度无效
    """
    # 输入验证
    if not 0 <= percent_solids <= 100:
        raise ValueError("固体百分比必须在0-100之间")
    if ore_density <= 0 or water_density <= 0:
        raise ValueError("密度必须为正数")
    
    # 计算质量分数
    w_solids = percent_solids / 100
    w_water = 1 - w_solids
    
    # 计算倒数密度
    inverse_density = (w_solids / ore_density) + (w_water / water_density)
    
    # 返回矿浆密度
    return 1 / inverse_density

In [22]:
def calculate_mill_operation(
    D_ft: float,            # 磨机有效直径 (ft)
    L_ft: float,            # 磨机有效长度 (ft)
    Nc_pct: float,          # 转速 (%临界转速)
    J: float,               # 总填充率 (小数)
    Jb: float,              # 钢球填充率 (小数)
    Jp: float,              # 间隙矿浆填充率 (小数)
    fs: float,              # 矿浆固体百分比 (小数)
    rb: float,              # 钢球密度 (ton/m3)
    rm: float,              # 矿石密度 (ton/m3)
    alpha_deg: float,       # 提升角 (度)
    fv: float = 0.4,        # 钢球间隙率 (默认0.4)
    eta: float = 0.9        # 传动效率 (默认0.9)
) -> dict:
    """
    综合计算磨机装料参数和功率特性
    
    Returns:
        {
            'charge_parameters': {
                'mill_volume': 磨机总体积(m3),
                'apparent_volume': 表观装料体积(m3),
                'charge_weight': {
                    'total': 总重量(ton),
                    'balls': 钢球重量(ton),
                    'interstitial_slurry': 间隙矿浆重量(ton),
                    'overfilling_slurry': 溢流矿浆重量(ton)
                },
                'apparent_density': 表观密度(ton/m3)
            },
            'power_parameters': {
                'net_power': 净功率(kW),
                'gross_power': 总功率(kW),
                'power_breakdown': {
                    'balls': 钢球贡献功率(kW),
                    'interstitial_slurry': 间隙矿浆贡献功率(kW),
                    'overfilling_slurry': 溢流矿浆贡献功率(kW)
                },
                'intermediate_values': {
                    'critical_speed': 临界转速(rpm),
                    'actual_speed': 实际转速(rpm),
                    'slurry_density': 矿浆密度(ton/m3)
                }
            }
        }
    """
    # ========== 1. 单位转换 ==========
    D = D_ft * 0.305  # ft → m
    L = L_ft * 0.305
    
    # ========== 2. 磨机体积计算 ==========
    V_mill = np.pi * (D/2)**2 * L
    
    # ========== 3. 矿浆密度计算 ==========
    rp = 1 / (fs/rm + (1-fs)/1.0)  # 水密度=1.0
    
    # ========== 4. 装料参数计算 ==========
    # 各组分的质量(ton)
    M_balls = (1-fv) * rb * Jb * V_mill
    M_inter = rp * Jp * fv * Jb * V_mill
    M_over = rp * (J-Jb) * V_mill
    M_total = M_balls + M_inter + M_over
    
    # 表观体积和密度
    V_apparent = J * V_mill
    rap = M_total / V_apparent
    
    # ========== 5. 功率计算 ==========
    # 临界转速 (rpm)
    Ncrit = 76.6 / np.sqrt(D_ft)
    
    # 实际转速 (小数 fraction of critical)
    Nc = Nc_pct / 100
    
    # 净功率 (kW)
    Pnet = 0.238 * (D_ft**3.5) * (L_ft/D_ft) * Nc * rap * (J - 1.065*J**2) * np.sin(np.radians(alpha_deg))
    
    # 总功率 (kW)
    Pgross = Pnet / eta
    
    # 功率组分分解
    Pb = ((1-fv)*rb*Jb) / (rap*J) * (eta*Pgross)
    Ps = (rp*Jp*fv*Jb) / (rap*J) * (eta*Pgross)
    Po = (rp*(J-Jb)) / (rap*J) * (eta*Pgross)
    
    return {
        'charge_parameters': {
            'mill_volume': V_mill,
            'apparent_volume': V_apparent,
            'charge_weight': {
                'total': M_total,
                'balls': M_balls,
                'interstitial_slurry': M_inter,
                'overfilling_slurry': M_over
            },
            'apparent_density': rap
        },
        'power_parameters': {
            'net_power': Pnet,
            'gross_power': Pgross,
            'power_breakdown': {
                'balls': Pb,
                'interstitial_slurry': Ps,
                'overfilling_slurry': Po
            },
            'intermediate_values': {
                'critical_speed': Ncrit,
                'actual_speed': Ncrit * Nc,
                'slurry_density': rp
            }
        }
    }

### 磨机功率和过程参数

In [23]:

params = {
    'D_ft': 18.5,
    'L_ft': 22.0,
    'Nc_pct': 72.0,
    'J': 0.38,
    'Jb': 0.38,
    'Jp': 1.0,
    'fs': 0.72,
    'rb': 7.75,
    'rm': 2.8,
    'alpha_deg': 35.0
}

results = calculate_mill_operation(**params)

# 装料参数
charge = results['charge_parameters']
print("===== 装料参数 =====")
print(f"磨机总体积: {charge['mill_volume']:.2f} m3")
print(f"表观装料体积: {charge['apparent_volume']:.2f} m3")
print(f"钢球重量: {charge['charge_weight']['balls']:.2f} ton")
print(f"间隙矿浆重量: {charge['charge_weight']['interstitial_slurry']:.2f} ton")
print(f"溢流矿浆重量: {charge['charge_weight']['overfilling_slurry']:.2f} ton")
print(f"总重量: {charge['charge_weight']['total']:.2f} ton")
print(f"表观密度: {charge['apparent_density']:.3f} ton/m3")

# 功率参数
power = results['power_parameters']
print("\n===== 功率参数 =====")
print(f"临界转速: {power['intermediate_values']['critical_speed']:.2f} rpm")
print(f"实际转速: {power['intermediate_values']['actual_speed']:.2f} rpm")
print(f"矿浆密度: {power['intermediate_values']['slurry_density']:.3f} ton/m3")
print(f"\n净功率: {power['net_power']:.1f} kW")
print(f"总功率: {power['gross_power']:.1f} kW")
print("\n功率分布:")
print(f"- 钢球贡献: {power['power_breakdown']['balls']:.1f} kW ({power['power_breakdown']['balls']/power['net_power']*100:.1f}%)")
print(f"- 间隙矿浆贡献: {power['power_breakdown']['interstitial_slurry']:.1f} kW ({power['power_breakdown']['interstitial_slurry']/power['net_power']*100:.1f}%)")
print(f"- 溢流矿浆贡献: {power['power_breakdown']['overfilling_slurry']:.1f} kW ({power['power_breakdown']['overfilling_slurry']/power['net_power']*100:.1f}%)")

===== 装料参数 =====
磨机总体积: 167.79 m3
表观装料体积: 63.76 m3
钢球重量: 296.48 ton
间隙矿浆重量: 47.48 ton
溢流矿浆重量: 0.00 ton
总重量: 343.96 ton
表观密度: 5.395 ton/m3

===== 功率参数 =====
临界转速: 17.81 rpm
实际转速: 12.82 rpm
矿浆密度: 1.862 ton/m3

净功率: 3884.5 kW
总功率: 4316.1 kW

功率分布:
- 钢球贡献: 3348.3 kW (86.2%)
- 间隙矿浆贡献: 536.2 kW (13.8%)
- 溢流矿浆贡献: 0.0 kW (0.0%)


### 选择函数和破碎函数

In [6]:
params = read_grinding_config('grinding_config.ini', flatten=True)


In [14]:
dcrit = params['dcrit']
alpha01 = alpha0 = params['alpha0']
alpha11 = alpha1 = params['alpha1']
alpha2 = params['alpha2']

alpha02 = params['alpha02']
alpha12 = params['alpha12']

beta00 = params['beta0']
beta01 = params['beta01']
beta1 = params['beta1']
beta2 = params['beta2']

ore_density_mill = params['ore (ton/m3)']
solids_mill = params['mill discharge (%)']
ball_dnesity_mill = params['balls (ton/m3)']

In [15]:
ball_dnesity_mill

7.75

In [13]:
calculate_slurry_density(
    percent_solids=solids_mill,
    ore_density=ore_density_mill,
    water_density=1.00
)


1.8617021276595744

In [54]:
siE_array = np.zeros(5)
bij_array = np.zeros((5, 5))

for i in range(len(siE_array)):
    d_i = float(params['mesh_'+str(i+1)]['opening'])
    d_ip1 = float(params['mesh_' + str(i+2)]['opening'])
    d_i_star = np.sqrt(d_i*d_ip1)
    
    siE = selection_function_expanded(d_i_star, alpha01, alpha11, alpha02, alpha12, dcrit, alpha2)
    siE_array[i] = siE

    for j in range(len(siE_array)):
        if not j > i:
            d_j_plus_1 = float(params['mesh_' + str(j+2)]['opening'])
            bij = breakage_function_expanded(d_ip1, d_j_plus_1, beta00, beta01, beta1, beta2)
            bij_array[i][j] = bij

print("siE array:", siE_array)
print("bij array:\n", bij_array)

siE array: [0.27970586 0.49950547 0.83236684 1.19137926 1.48778328]
bij array:
 [[1.         0.         0.         0.         0.        ]
 [0.33874509 1.         0.         0.         0.        ]
 [0.21754595 0.43647669 1.         0.         0.        ]
 [0.16626002 0.23241927 0.38120347 1.         0.        ]
 [0.14442079 0.17206053 0.21817928 0.38562004 1.        ]]


### J&T